In [1]:
import numpy as np
from scipy import stats
import unittest
from math import gamma as gamma_fn
%run helper_fns.ipynb

## Probability functions and checks
This notebook is to be used as a submodule that contains wrappers for all the basic probabilty functions used by the ordinal probit model for survey data, and Metropolis-Hastings sampler. There is also an optional testing suite.

In [23]:
def mu_prior(mu, mu_0, sigma_0, printing=False, debug=False):
    """Wrapper for prior on mu. Distribution and hyperparameters taken from original paper.
    
    Inputs
    -------------------
    mu: latent mean; scalar value in Reals;
    mu_0: prior mean; scalar value in Reals;
    sigma_0: prior standard devation; scalar value in (0, inf);
    printing: Bool; whether to print messages;
    debug: Bool; whether to run internal error checks.
    
    Outputs
    -------------------
    p: prior probability of the input value of mu."""
    
    p = stats.norm.pdf(mu, mu_0, sigma_0)
    
    if debug:
        assert 0 < p < 1, "p not in right range: {}".format(p)
        assert isinstance(p, float) is True, "type of p is not scalar: {}".format(type(p))
    return p

def mus_log_prior(mus, mu_0, sigma_0, printing=False, debug=False):
    """Returns log prior over all mus. 
    Procedure, distributions and hyperparameters taken from original paper.
    
    Inputs
    -------------------
    mus: vector of scalar value in Reals corresponding to mus;
    mu_0: prior mean; scalar value in Reals;
    sigma_0: prior standard devation for all mus; scalar value in (0, inf);
    printing: Bool; whether to print messages;
    debug: Bool; whether to run internal error checks.
    
    Outputs
    -------------------
    LP: log prior probability of the input value of mus."""
    
    LP = 0 # log prior
    
    for mu in mus: # ignore first and last value, as these aren't really variables
        LP += np.log(mu_prior(mu, mu_0, sigma_0, printing, debug))
    if debug:
        assert isinstance(LP, float) is True, "type of LP is not scalar: {}".format(type(LP))
            
    return LP

def mu_proposal(mu, sigma_prop, printing=False, debug=False):
    """Function for Metropolis sampler to generate proposal for individual mu. 
    Simple random walk. 
    
    Inputs
    -------------------
    mu: latent mean; scalar value in Reals;
    sigma_prop: standard deviation of jumps; 
    printing: Bool; whether to print messages;
    debug: Bool; whether to run internal error checks.
    
    Outputs
    -------------------
    mu_star: new proposed value for mu."""
    
    mu_star = np.random.normal(mu, sigma_prop)
    
    if debug:
        assert isinstance(mu_star, float) is True, "type of LP is not scalar: {}".format(type(mu_star))
        
    return mu_star

def mu_log_jump_probs(mu, mu_star, sigma_prop, printing=False, debug=False):
    """While using a symmetric (i.e., non-truncated) proposal, this is merely"""
    return 0 

def convert_gamma_parameters(gamma_mean, gamma_spread, printing=False, debug=False):
    """Helper function for dealing with gamma distribution, hyperprior and proposal distribution
    for sigma."""
    alpha = (gamma_mean**2) / (gamma_spread**2) # shape param, algebra from wikipedia
    beta = gamma_mean / (gamma_spread**2) # algebra from wikipedia
    scale = 1/beta
    
    return alpha, scale

def sigma_prior(sigma, gamma_mean, gamma_spread, printing=False, debug=False):
    """Wrapper for prior on sigma. Distribution and hyperparameters taken from original paper.
    
    Inputs
    -------------------
    sigma: scalar value in (0, inf);
    gamma_mean: float, mean of gamma prior for sigma;
    gamma_spread: float from positive Reals, sd of gamma prior for sigma; 
    printing: Bool; whether to print messages;
    debug: Bool; whether to run internal error checks.
    
    Outputs
    -------------------
    p: prior probability of the input value of sigma."""
    alpha, scale = convert_gamma_parameters(gamma_mean, gamma_spread)
    p = stats.gamma.pdf(sigma, alpha, scale=scale) # I think this is right; loc not needed here
    if printing:
        print_fn(['sigma', sigma, 'gamma_mean', gamma_mean, 'gamma_spread', gamma_spread, 
                  'alpha', alpha, 'scale', scale,
                 'p', p])
    if debug:
        assert sigma > 0, " sigma is negative: {}".format(sigma) # this will be caught anyway by above line
        assert 0 <= p <= 1, "p not in right range: {}".format(p)
        assert isinstance(p, float) is True, "type of p is not scalar: {}".format(type(p))
    return p

def sigmas_log_prior(sigmas, gamma_mean, gamma_spread, printing=False, debug=False):
    """Returns log prior over all sigmas. 
    Procedure, distributions and hyperparameters taken from original paper.
    
    Inputs
    -------------------
    sigmas: vector of scalar values in (0, inf);
    gamma_mean: float, mean of gamma prior for sigma;
    gamma_spread: float from positive Reals, sd of gamma prior for sigma; 
    printing: Bool; whether to print messages;
    debug: Bool; whether to run internal error checks.
    
    Outputs
    -------------------
    LP: log prior probability of the input value of sigmas."""
    
    probs = [] # log prior
    
    for sigma in sigmas: # ignore first and last value, as these aren't really variables
        probs.append(sigma_prior(sigma, gamma_mean, gamma_spread, printing, debug))
    LP = np.sum(np.log(probs))
    
    if debug:
        assert isinstance(LP, float), "type of LP is not scalar: {}".format(type(LP))
    if printing:
        print_fn(['sigmas', sigmas, 'gamma_mean', gamma_mean, 'gamma_spread', gamma_spread, 
                 'probs', probs, 'LP', LP])
    return LP

# needs work
def sigma_proposal(sigma, gamma_spread, upper_0, printing=False, debug=False):
    """Function for Metropolis sampler to generate proposal for individual sigma. 
    Simple random walk that reflects on both sides of interval.
    
    Inputs
    -------------------
    sigma: current latent standard deviation; scalar value in (0, inf); used as mean for new distribution
    gamma_spread: scale of jumps; scalar in positive reals;
    printing: Bool; whether to print messages;
    debug: Bool; whether to run internal error checks.
    
    Outputs
    -------------------
    sigma_star: new proposed value for sigma."""
    
    alpha, scale = convert_gamma_parameters(sigma, gamma_spread)
    sigma_star = stats.gamma.rvs(alpha, scale=scale)
    if debug:
        assert sigma_star > 0, "sigma proposal nonpositive: {}".format(sigma_star)
    return sigma_star

# needs work
def sigma_proposal_prob(sigma_new, sigma_old, gamma_spread, printing=False, debug=False):
    """Function for Metropolis-Hastings sampler to evaluate proposal for individual sigma. 
    Simple random walk that reflects on both sides of interval. Note, have to convert lower and upper
    to standard normal form; scipy then reconverts based on scale and location parameters.
    
    Inputs
    -------------------
    gamma_mean: float, mean of gamma prior for sigma;
    gamma_spread: float from positive Reals, sd of gamma prior for sigma; 
    sigma_prop: scale of jumps; 
    printing: Bool; whether to print messages;
    debug: Bool; whether to run internal error checks.
    
    Outputs
    -------------------
    p: probability for value of sigma."""
    alpha, scale = convert_gamma_parameters(sigma_old, gamma_spread)
    p = stats.gamma.pdf(sigma_new, alpha, scale=scale)
    if debug:
        assert 0 <= p <= 1, "p not in right range: {}".format(p)
        assert isinstance(p, float) is True, "type of p is not scalar: {}".format(type(p))
    return p

# arguments need updating
def sigma_log_jump_probs(sigma, sigma_star, gamma_spread, printing=False, debug=False):
    """If using truncated normal, no longer symmetric. This is now Metropolis-Hastings algorithm, 
    and we have to add log of ratio of proposal probabilities to log of acceptance probabilities.
    
        Inputs
    -------------------
    sigma: latent standard deviation; scalar value in (lower, upper);
    sigma_star: proposed value for latent standard deviation; scalar value in (lower, upper);
    gamma_spread: float from positive Reals, sd of gamma proposal jump for sigma;
    printing: Bool; whether to print messages;
    debug: Bool; whether to run internal error checks.
    
    Outputs
    -------------------
    LP: log probability of ratio of jump probabilities."""
    # first argument is proposal, second mean
    LP = np.log(sigma_proposal_prob(sigma, sigma_star, gamma_spread, printing, debug)) - np.log(sigma_proposal_prob(sigma_star, sigma, gamma_spread, printing, debug))
        
    if debug:
        assert isinstance(LP, float) is True, "type of LP is not scalar: {}".format(type(LP))
        
    return LP

def theta_prior(theta, mu_0, sigma_0, printing=False, debug=False):
    """Wrapper for prior on theta. Distribution and hyperparameters taken from original paper.
    
    Inputs
    -------------------
    theta: scalar value in Reals;
    mu_0: prior mean; scalar value in Reals;
    sigma_0: prior standard devation; scalar value in (0, inf);
    printing: Bool; whether to print messages;
    debug: Bool; whether to run internal error checks.
    
    Outputs
    -------------------
    p: prior probability of the input value of theta."""
    
    p = stats.norm.pdf(theta, mu_0, sigma_0)
    
    if debug:
        assert 0 < p < 1, "p not in right range: {}".format(p)
        assert isinstance(p, float) is True, "type of p is not scalar: {}".format(type(p))
    if printing:
        print("in theta prior")
        print_fn(["theta", theta, "mu_0", mu_0, sigma_0, "sigma_0"])
    return p

def thetas_log_prior(thetas, shift, sigma_0, printing=False, debug=False):
    """Returns log prior over all thetas. 
    Procedure, distributions and hyperparameters taken from original paper.
    
    Inputs
    -------------------
    thetas: vector of scalar value in Reals corresponding to thetas;
    shift: value to shift prior means by; scalar value in Reals;
    sigma_0: prior standard devation for all thetas; scalar value in (0, inf);
    printing: Bool; whether to print messages;
    debug: Bool; whether to run internal error checks.
    
    Outputs
    -------------------
    LP: log prior probability of the input value of thetas."""
    
    k = len(thetas) - 2 # thetas have endpoints of -inf and inf
    thetaPriors = []
    ks = []
    
    
    for k in np.arange(1,k+1): # ignore first and last value, as these aren't really variables
        ks.append(k+shift)
        thetaPriors.append(theta_prior(thetas[k], k+shift, sigma_0, printing, debug))
    LP = np.sum(np.log(thetaPriors))
    
    if debug:
        assert isinstance(LP, float) is True, "type of LP is not scalar: {}".format(type(LP))
        test = np.append(thetas[1:], 1) - thetas < 0
        if test[:-1].any():
            print("some thetas out of order: {}; test: {}".format(thetas, test))
    if printing:
            print_fn(["sigma_0", sigma_0])
            print_fn(["ks plus shift", ks])
            print_fn(["thetas", thetas])
            print_fn(["thetaPriors", thetaPriors])
            print_fn(["theta log prior internal", LP])
    return LP

def theta_proposal(theta, sigma_prop, lower_0, upper_0, printing=False, debug=False):
    """Function for Metropolis sampler to generate proposal for individual theta. 
    Simple random walk that reflects on both sides of interval.
    
    Inputs
    -------------------
    theta: latent threshold; scalar value in (lower_0, upper_0); typically (0, k)
    sigma_prop: scale of jumps; 
    lower_0: lower end of range;
    upper_0: upper end of range; 
    printing: Bool; whether to print messages;
    debug: Bool; whether to run internal error checks.
    
    Outputs
    -------------------
    theta_star: new proposed value for theta."""
    
    theta_star = stats.truncnorm.rvs(lower_0, upper_0, theta, sigma_prop)
    if debug:
        assert lower_0 <= theta_star <= upper_0
    return sigma_star

def theta_proposal_prob(theta_new, theta_old, sigma_prop, lower_0, upper_0, printing=False, debug=False):
    """Function for Metropolis-Hastings sampler to evaluate proposal for individual theta. 
    Simple random walk that reflects on both sides of interval. Note, have to convert lower and upper
    to standard normal form; scipy then reconverts based on scale and location parameters.
    
    Inputs
    -------------------
    theta_old: latent threshold; scalar value in (lower_0, upper_0); typically (0, k)
    theta_new: latent threshold; scalar value in (lower_0, upper_0); typically (0, k)
    sigma_prop: scale of jumps; 
    lower_0: lower end of uniform;
    upper_0: upper end of uniform; 
    printing: Bool; whether to print messages;
    debug: Bool; whether to run internal error checks.
    
    Outputs
    -------------------
    p: probability for value of theta."""
    a = (lower_0 - theta_old)/sigma_prop
    b = (upper_0 - theta_old)/sigma_prop
    p = stats.truncnorm.pdf(theta_new, a, b, theta_old, sigma_prop)
    if debug:
        assert 0 <= p <= 1, "p not in right range: {}".format(p)
        assert isinstance(p, float) is True, "type of p is not scalar: {}".format(type(p))
    return p
    

def theta_log_jump_probs(theta, theta_star, sigma_prop, lower_0, upper_0, printing=False, debug=False):
    """If using truncated normal, no longer symmetric. This is now Metropolis-Hastings algorithm, 
    and we have to add log of ratio of proposal probabilities to log of acceptance probabilities.
    
        Inputs
    -------------------
    theta: latent threshold; scalar value in (lower_0, upper_0); typically (0, k)
    theta_star: latent threshold proposal; scalar value in (lower_0, upper_0); typically (0, k)
    sigma_prop: scale of jumps; 
    lower_0: lower end of uniform;
    upper_0: upper end of uniform; 
    printing: Bool; whether to print messages;
    debug: Bool; whether to run internal error checks.
    
    Outputs
    -------------------
    LP: log probability of ratio of jump probabilities."""
    # first argument is new proposal
    first_term = theta_proposal_prob(theta, theta_star, sigma_prop, 
                                    lower_0, upper_0, printing, debug)
    second_term = theta_proposal_prob(theta_star, theta, sigma_prop, 
                                      lower_0, upper_0, printing, debug)
    LP = np.log(first_term) - np.log(second_term)
    
#     if printing:
#         print("First theta log jump probs is {}; second is {}; log diff is {}".format(first_term, 
#                                                                                       second_term, LP))
    if debug:
        assert isinstance(LP, float) is True, "type of LP is not scalar: {}".format(type(LP))
        
    return LP

class ProbabilityFunctionsTestSuite(unittest.TestCase):
    
    def gauss(self, val, mu=0, s=1):
            return np.exp(- (1/2)*
                          ((val-mu)/s)**2) / np.sqrt(2*np.pi*(s**2))
    
    def gauss_trunc(self, x, mu, sigma, lower, upper):
        zeta = (x-mu)/sigma
        alpha = (lower-mu)/sigma
        beta = (upper-mu)/sigma
        Z = stats.norm.cdf(beta) - stats.norm.cdf(alpha)
        return (self.gauss(zeta)) / (sigma*Z)
    
    def test_mu_prior(self):
        """Test mu_prior"""
        testValues = [0, 0, 1]
        returned_p = mu_prior(*testValues, printing=False, debug=True)
        real_p = self.gauss(*testValues)
        self.assertAlmostEqual(returned_p, real_p)
        
    def test_mus_log_prior(self):
        """Test mus_log_prior"""
        testValues = [[0, 1], 0, 1]
        returned_LP = mus_log_prior(*testValues, printing=False, debug=True)
        probs = [self.gauss(x, testValues[1], testValues[2]) for x in testValues[0]]
        real_LP = np.sum(np.log(probs))
        self.assertAlmostEqual(returned_LP, real_LP)
        
    # this will need changing
    def test_sigma_prior(self):
        """Test sigma_prior"""
        testValues = [1.0, 2.0, 2.0]
        real_p = self.gamma(*testValues, printing=False)
        returned_p = sigma_prior(*testValues, printing=False, debug=True)
        
        self.assertAlmostEqual(returned_p, real_p)
        mean = 2.0
        sd = 3.0
        testValues2 = [0.1, mean, sd]
#         print('exterior mean is {}, spread is {}'.format(mean, sd))
        real_p = self.gamma(*testValues2, printing=False)
#         print('interior alpha is {}; scale is {}'.format(*convert_gamma_parameters(mean, sd)))
#         print('exterior p: {}'.format(real_p))
        returned_p = sigma_prior(*testValues2, printing=False, debug=True)
#         print('returned p', returned_p)
        
        self.assertAlmostEqual(returned_p, real_p)
        
    # this will need changing
    def test_sigmas_log_prior(self):
        """Test sigmas_log_prior"""
        testValues = [[1, 0.2], 0.5, 1.5]
        returned_LP = sigmas_log_prior(*testValues, printing=False, debug=True)

        probs = [self.gamma(x, 0.5, 1.5) for x in testValues[0]]
#         print('exterior probs has value; {}'.format(probs))
        real_LP = np.sum(np.log(probs))
        self.assertAlmostEqual(returned_LP, real_LP)
        
    def gamma(self, x, gamma_mean, gamma_spread, printing=False):
        """Return probability under gamma distribution."""
        alpha = (gamma_mean**2) / (gamma_spread**2) # shape param, algebra from wikipedia
        beta = gamma_mean / (gamma_spread**2) # algebra from wikipedia
        
        term_1 = (beta**alpha)
        term_2 = 1/gamma_fn(alpha)
 
        term_3 = (x**(alpha-1)) 
        term_4 = np.exp(-beta*x)
        if printing:
            print('exterior alpha, beta', alpha, beta)
        #print_fn(["term_1", term_1, "term_2", term_2, "term_3", term_3, "term_4", term_4])
        return term_1*term_2*term_3*term_4
    
    def test_scipy_gamma(self):
        """alpha and beta (rate) are Bayesian way of parameterizing."""
        alpha = 2
        beta = 2
        #print('alpha, beta', alpha, beta)
        mean = alpha/beta
        sd = np.sqrt(alpha)/beta
        alpha, inv_beta = convert_gamma_parameters(mean, sd, debug=True)
        #print('alpha', alpha, 'beta', 1/inv_beta)
        p = stats.gamma.pdf(mean, alpha, scale=inv_beta)
        gamma_p = self.gamma(mean, mean, sd)
        self.assertAlmostEqual(p, gamma_p)
        
        
    def test_sigma_proposal_prob(self):
        """Test sigma_proposal_prob; https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gamma.html;"""
        testValues = [1, 2, 3.0] # proposal, old, gamma_spread
        returned_p = sigma_proposal_prob(*testValues, printing=False, debug=True)
        
        # this will not be true
        real_p = self.gamma(*testValues)
        self.assertAlmostEqual(returned_p, real_p)
        
    # this will need changing
    def test_sigma_log_jump_probs(self):
        testValues1 = [1, 2, 3.0]
        testValues2 = [2, 1, 3.0]
        returned_LP = sigma_log_jump_probs(*testValues1, printing=False, debug=True)
        
        # below will not be right, and needs changing
        real_LP = np.log(self.gamma(*testValues1)) - np.log(self.gamma(*testValues2)) 
        self.assertAlmostEqual(returned_LP, real_LP)
        
    def test_theta_prior(self):
        """Test theta_prior"""
        testValues = [0, 0, 1]
        returned_p = theta_prior(*testValues, printing=False, debug=True)
        real_p = self.gauss(*testValues)
        self.assertAlmostEqual(returned_p, real_p)
        
    def test_thetas_log_prior(self):
        """Test thetas_log_prior"""
        testValues = [[-np.inf, 1.5, 2.5, 3.5, np.inf], 0.5, 1.0]
        returned_LP = thetas_log_prior(*testValues, printing=False, debug=False)
        probs = [self.gauss(x, i+1.5, 1) for (i, x) in enumerate([1.5, 2.5, 3.5])]
        real_LP = np.sum(np.log(probs))
        self.assertAlmostEqual(returned_LP, real_LP)
        
    def test_theta_proposal_prob(self):
        """Test theta_proposal_prob; https://en.wikipedia.org/wiki/Truncated_normal_distribution"""
        testValues = [1, 2, 1, 0, 3]
        returned_p = theta_proposal_prob(*testValues, printing=False, debug=True)
        real_p = self.gauss_trunc(*testValues)
        self.assertAlmostEqual(returned_p, real_p)
        
    def test_theta_log_jump_probs(self):
        testValues1 = [1, 2, 1, 0, 3]
        testValues2 = [2, 1, 1, 0, 3]
        returned_LP = theta_log_jump_probs(*testValues1, printing=False, debug=False)
        real_LP = np.log(self.gauss_trunc(*testValues2)) - np.log(self.gauss_trunc(*testValues1))
        self.assertAlmostEqual(returned_LP, real_LP)
        
runner = unittest.TextTestRunner(failfast=True)
runner.run(initialize_suite(ProbabilityFunctionsTestSuite))
### add test for above

...........
----------------------------------------------------------------------
Ran 11 tests in 0.010s

OK


<unittest.runner.TextTestResult run=11 errors=0 failures=0>